In [20]:
#import statements
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as ply
import seaborn as sns

#sci-kit learn
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Data Cleaning and EDA

In [21]:
#import train data
#DO NOT LOOK AT TEST DATA UNTIL VALIDATION
df_train = pd.read_csv('../data/water_well_train_data.csv')

In [22]:
df_train.head()

id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572      6000.0    2011-03-14         Roman        1390         Roman   
1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
4  19728         0.0    2011-07-13   Action In A           0       Artisan   

   longitude   latitude              wpt_name  num_private  ... payment_type  \
0  34.938093  -9.856322                  none            0  ...     annually   
1  34.698766  -2.147466              Zahanati            0  ...    never pay   
2  37.460664  -3.821329           Kwa Mahundi            0  ...   per bucket   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0  ...    never pay   
4  31.130847  -1.825359               Shuleni            0  ...    never pay   

  water_quality quality_group      quantity  quantity_group  \
0          soft          good        enough          enough   
1          soft          good  insufficient    insufficient   
2          soft          good        enough          enough   
3          soft          good           dry             dry   
4          soft          good      seasonal        seasonal   

                 source           source_type  source_class  \
0                spring                spring   groundwater   
1  rainwater harvesting  rainwater harvesting       surface   
2                   dam                   dam       surface   
3           machine dbh              borehole   groundwater   
4  rainwater harvesting  rainwater harvesting       surface   

               waterpoint_type waterpoint_type_group  
0           communal standpipe    communal standpipe  
1           communal standpipe    communal standpipe  
2  communal standpipe multiple    communal standpipe  
3  communal standpipe multiple    communal standpipe  
4           communal standpipe    communal standpipe  

[5 rows x 40 columns]

In [23]:
# info of train data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [24]:
#import target information
df_label = pd.read_csv('../data/water_well_train_labels.csv')

In [25]:
df_label.shape

(59400, 2)

In [26]:
df_label.head()

id    status_group
0  69572      functional
1   8776      functional
2  34310      functional
3  67743  non functional
4  19728      functional

In [27]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [28]:
df_label['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [29]:
df_label['status_group'].value_counts(normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

There are three target classifications: functional (54%), non-functional (38%), and function needs repair (7%).

In [30]:
#combine train and label dataframes prior to cleaning to address any dropped rows
df = df_train.join(other = df_label, rsuffix = '_label')

In [31]:
df.head()

id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572      6000.0    2011-03-14         Roman        1390         Roman   
1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
4  19728         0.0    2011-07-13   Action In A           0       Artisan   

   longitude   latitude              wpt_name  num_private  ... quality_group  \
0  34.938093  -9.856322                  none            0  ...          good   
1  34.698766  -2.147466              Zahanati            0  ...          good   
2  37.460664  -3.821329           Kwa Mahundi            0  ...          good   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0  ...          good   
4  31.130847  -1.825359               Shuleni            0  ...          good   

       quantity quantity_group                source           source_type  \
0        enough         enough                spring                spring   
1  insufficient   insufficient  rainwater harvesting  rainwater harvesting   
2        enough         enough                   dam                   dam   
3           dry            dry           machine dbh              borehole   
4      seasonal       seasonal  rainwater harvesting  rainwater harvesting   

  source_class              waterpoint_type  waterpoint_type_group id_label  \
0  groundwater           communal standpipe     communal standpipe    69572   
1      surface           communal standpipe     communal standpipe     8776   
2      surface  communal standpipe multiple     communal standpipe    34310   
3  groundwater  communal standpipe multiple     communal standpipe    67743   
4      surface           communal standpipe     communal standpipe    19728   

     status_group  
0      functional  
1      functional  
2      functional  
3  non functional  
4      functional  

[5 rows x 42 columns]

In [32]:
#check that id columns align
df[df['id'] != df['id_label']]

Empty DataFrame
Columns: [id, amount_tsh, date_recorded, funder, gps_height, installer, longitude, latitude, wpt_name, num_private, basin, subvillage, region, region_code, district_code, lga, ward, population, public_meeting, recorded_by, scheme_management, scheme_name, permit, construction_year, extraction_type, extraction_type_group, extraction_type_class, management, management_group, payment, payment_type, water_quality, quality_group, quantity, quantity_group, source, source_type, source_class, waterpoint_type, waterpoint_type_group, id_label, status_group]
Index: []

[0 rows x 42 columns]

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Columns with nulls:
- funder
- installer
- subvillage
- public_meeting
- scheme_management
- scheme_name
- permit

In [34]:
df.describe()

id     amount_tsh    gps_height     longitude      latitude  \
count  59400.000000   59400.000000  59400.000000  59400.000000  5.940000e+04   
mean   37115.131768     317.650385    668.297239     34.077427 -5.706033e+00   
std    21453.128371    2997.574558    693.116350      6.567432  2.946019e+00   
min        0.000000       0.000000    -90.000000      0.000000 -1.164944e+01   
25%    18519.750000       0.000000      0.000000     33.090347 -8.540621e+00   
50%    37061.500000       0.000000    369.000000     34.908743 -5.021597e+00   
75%    55656.500000      20.000000   1319.250000     37.178387 -3.326156e+00   
max    74247.000000  350000.000000   2770.000000     40.345193 -2.000000e-08   

        num_private   region_code  district_code    population  \
count  59400.000000  59400.000000   59400.000000  59400.000000   
mean       0.474141     15.297003       5.629747    179.909983   
std       12.236230     17.587406       9.633649    471.482176   
min        0.000000      1.000000       0.000000      0.000000   
25%        0.000000      5.000000       2.000000      0.000000   
50%        0.000000     12.000000       3.000000     25.000000   
75%        0.000000     17.000000       5.000000    215.000000   
max     1776.000000     99.000000      80.000000  30500.000000   

       construction_year      id_label  
count       59400.000000  59400.000000  
mean         1300.652475  37115.131768  
std           951.620547  21453.128371  
min             0.000000      0.000000  
25%             0.000000  18519.750000  
50%          1986.000000  37061.500000  
75%          2004.000000  55656.500000  
max          2013.000000  74247.000000

## Dealing with Nulls

### funder

In [35]:
#who funded the well
print(df['funder'].isna().sum())
df['funder'].value_counts()

3635


Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
                          ... 
Hasnein Muij Mbunge          1
Nyangere                     1
People Of Sweden             1
Ccpk                         1
Magu Food Security           1
Name: funder, Length: 1897, dtype: int64

In [36]:
unique_funder = list(df['funder'].unique())
print(len(unique_funder))
unique_funder

1898


['Roman',
 'Grumeti',
 'Lottery Club',
 'Unicef',
 'Action In A',
 'Mkinga Distric Coun',
 'Dwsp',
 'Rwssp',
 'Wateraid',
 'Isingiro Ho',
 'Private',
 'Danida',
 'World Vision',
 'Lawatefuka Water Supply',
 'Biore',
 'Rudep',
 'Hesawa',
 'Twe',
 'Isf',
 'African Development Bank',
 'Government Of Tanzania',
 'Sobodo',
 'Water',
 'Private Individual',
 'Undp',
 nan,
 'Not Known',
 'Kirde',
 'Cefa',
 'Ces(gmbh)',
 'European Union',
 'Lga',
 'District Council',
 'Muwsa',
 'Dwe/norad',
 'Kkkt_makwale',
 'Sawaka',
 'Ces (gmbh)',
 'Olgilai Village Community',
 'Kkkt',
 'Roman Catholic',
 'Norad',
 'Adra',
 'Sema',
 'Piusi',
 'Dwe',
 'Rc Church',
 'Swisland/ Mount Meru Flowers',
 'Ifad',
 'Swedish',
 'Idc',
 'He',
 'Isf/tacare',
 'Jica',
 'Mzee Sh',
 'Aict',
 'Tcrs',
 'Kiuma',
 'Germany Republi',
 'Netherlands',
 'Ruthe',
 'Tulawaka Gold Mine',
 'Nethalan',
 'Tasaf',
 'Concern World Wide',
 'Wfp',
 'Lips',
 'Sida',
 'World Bank',
 'Tanza',
 '0',
 'Sw',
 'Shipo',
 'Fini Water',
 'Kanisa',
 'Ox

In [37]:
df[df['funder'] == '0']

id  amount_tsh date_recorded funder  gps_height installer  \
145    28669        50.0    2013-03-21      0          12         0   
152    60983         0.0    2013-03-16      0         -15         0   
393    39749         0.0    2013-03-18      0          28         0   
417    15832        50.0    2013-03-22      0          30         0   
428    50233         0.0    2013-03-12      0          30         0   
...      ...         ...           ...    ...         ...       ...   
59237   2138        59.0    2013-03-19      0          81         0   
59243   3396        50.0    2013-03-16      0         -20         0   
59276  62818        50.0    2013-03-21      0          18         0   
59351  55322        50.0    2013-03-18      0         -19         0   
59387  26640       100.0    2013-03-12      0          25         0   

       longitude  latitude       wpt_name  num_private  ... quality_group  \
145    39.185710 -6.892593       Chekanao            0  ...          good   
152    39.527114 -6.988748      Msikitini            0  ...         salty   
393    39.159887 -6.902548   Kwa Chambuso            0  ...       unknown   
417    39.178404 -6.938013     Ccm Kivule            0  ...          good   
428    39.178849 -6.973206  Ofisi Ya Kata            0  ...       unknown   
...          ...       ...            ...          ...  ...           ...   
59237  39.119109 -6.898919           Kata            0  ...         salty   
59243  39.524021 -6.984802   Kwa Mariwala            0  ...          good   
59276  39.183790 -6.897566   Kwa Mkunduge            0  ...          good   
59351  39.534599 -7.088183      Kwa China            0  ...          good   
59387  39.176480 -6.957098     Kwa Maliba            0  ...          good   

           quantity quantity_group        source   source_type source_class  \
145          enough         enough   machine dbh      borehole  groundwater   
152             dry            dry   machine dbh      borehole  groundwater   
393         unknown        unknown   machine dbh      borehole  groundwater   
417          enough         enough   machine dbh      borehole  groundwater   
428             dry            dry  shallow well  shallow well  groundwater   
...             ...            ...           ...           ...          ...   
59237  insufficient   insufficient   machine dbh      borehole  groundwater   
59243        enough         enough  shallow well  shallow well  groundwater   
59276        enough         enough   machine dbh      borehole  groundwater   
59351        enough         enough   machine dbh      borehole  groundwater   
59387        enough         enough   machine dbh      borehole  groundwater   

          waterpoint_type  waterpoint_type_group id_label    status_group  
145    communal standpipe     communal standpipe    28669      functional  
152                 other                  other    60983  non functional  
393                 other                  other    39749  non functional  
417    communal standpipe     communal standpipe    15832  non functional  
428                 other                  other    50233  non functional  
...                   ...                    ...      ...             ...  
59237  communal standpipe     communal standpipe     2138  non functional  
59243           hand pump              hand pump     3396      functional  
59276  communal standpipe     communal standpipe    62818      functional  
59351  communal standpipe     communal standpipe    55322      functional  
59387  communal standpipe     communal standpipe    26640      functional  

[777 rows x 42 columns]

In [38]:
#replace nulls in 'funder' col with "Not Known"
df['funder'].fillna(value="Not Known", inplace=True)

In [39]:
#confirm replacement of nulls
df['funder'].isna().sum()

0

### installer

In [40]:
#organization that installed the well
df['installer'].value_counts()

DWE                     17402
Government               1825
RWE                      1206
Commu                    1060
DANIDA                   1050
                        ...  
magige                      1
Ester Ndege                 1
Steven Nyangarika           1
Jumanne                     1
Goldwill  foundation        1
Name: installer, Length: 2145, dtype: int64

In [41]:
df['installer'].isna().sum()

3655

In [42]:
list(df['installer'].unique())

['Roman',
 'GRUMETI',
 'World vision',
 'UNICEF',
 'Artisan',
 'DWE',
 'DWSP',
 'Water Aid',
 'Private',
 'DANIDA',
 'Lawatefuka water sup',
 'WEDECO',
 'Danid',
 'TWE',
 'ISF',
 'Kilolo Star',
 'District council',
 'Water',
 'WU',
 nan,
 'Not known',
 'Central government',
 'CEFA',
 'Commu',
 'Accra',
 'World Vision',
 'LGA',
 'MUWSA',
 'KKKT _ Konde and DWE',
 'Government',
 'Olgilai village community',
 'KKKT',
 'RWE',
 'Adra /Community',
 'SEMA',
 'SHIPO',
 'HESAWA',
 'ACRA',
 'Community',
 'IFAD',
 'Sengerema Water Department',
 'HE',
 'ISF and TACARE',
 'Kokeni',
 'DA',
 'Adra',
 'ALLYS',
 'AICT',
 'KIUMA',
 'CES',
 'District Counci',
 'Ruthe',
 'Adra/Community',
 'Tulawaka Gold Mine',
 'KKT C',
 'Hesawa',
 'Water board',
 'LOCAL CONTRACT',
 'WFP',
 'LIPS',
 'TASAF',
 'World',
 '0',
 'SW',
 'Shipo',
 'Fini water',
 'Kanisa',
 'OXFARM',
 'VILLAGE COUNCIL Orpha',
 'Villagers',
 'Idara ya maji',
 'FPCT',
 'WVT',
 'Ir',
 'DANID',
 'Angli',
 'secondary school',
 'Amref',
 'JBG',
 'DAD

In [43]:
df['installer'].fillna(value='Not known', inplace=True)

In [44]:
df['installer'].isna().sum()

0

In [45]:
#check cases where the installer is not also the funder
df[df['funder'] != df['installer']]

id  amount_tsh date_recorded               funder  gps_height  \
1       8776         0.0    2013-03-06              Grumeti        1399   
2      34310        25.0    2013-02-25         Lottery Club         686   
3      67743         0.0    2013-01-28               Unicef         263   
4      19728         0.0    2011-07-13          Action In A           0   
5       9944        20.0    2011-03-13  Mkinga Distric Coun           0   
...      ...         ...           ...                  ...         ...   
59394  11164       500.0    2011-03-09           World Bank         351   
59395  60739        10.0    2013-05-03      Germany Republi        1210   
59396  27263      4700.0    2011-05-07          Cefa-njombe        1212   
59398  31282         0.0    2011-03-08                Malec           0   
59399  26348         0.0    2011-03-23           World Bank         191   

          installer  longitude   latitude              wpt_name  num_private  \
1           GRUMETI  34.698766  -2.147466              Zahanati            0   
2      World vision  37.460664  -3.821329           Kwa Mahundi            0   
3            UNICEF  38.486161 -11.155298  Zahanati Ya Nanyumbu            0   
4           Artisan  31.130847  -1.825359               Shuleni            0   
5               DWE  39.172796  -4.765587                Tajiri            0   
...             ...        ...        ...                   ...          ...   
59394      ML appro  37.634053  -6.124830            Chimeredya            0   
59395           CES  37.169807  -3.253847   Area Three Namba 27            0   
59396          Cefa  35.249991  -9.070629     Kwa Yahona Kuvala            0   
59398          Musa  35.861315  -6.378573                Mshoro            0   
59399         World  38.104048  -6.747464       Kwa Mzee Lugawa            0   

       ... quality_group      quantity quantity_group                source  \
1      ...          good  insufficient   insufficient  rainwater harvesting   
2      ...          good        enough         enough                   dam   
3      ...          good           dry            dry           machine dbh   
4      ...          good      seasonal       seasonal  rainwater harvesting   
5      ...         salty        enough         enough                 other   
...    ...           ...           ...            ...                   ...   
59394  ...          good        enough         enough           machine dbh   
59395  ...          good        enough         enough                spring   
59396  ...          good        enough         enough                 river   
59398  ...          good  insufficient   insufficient          shallow well   
59399  ...         salty        enough         enough          shallow well   

                source_type source_class              waterpoint_type  \
1      rainwater harvesting      surface           communal standpipe   
2                       dam      surface  communal standpipe multiple   
3                  borehole  groundwater  communal standpipe multiple   
4      rainwater harvesting      surface           communal standpipe   
5                     other      unknown  communal standpipe multiple   
...                     ...          ...                          ...   
59394              borehole  groundwater           communal standpipe   
59395                spring  groundwater           communal standpipe   
59396            river/lake      surface           communal standpipe   
59398          shallow well  groundwater                    hand pump   
59399          shallow well  groundwater                    hand pump   

       waterpoint_type_group id_label    status_group  
1         communal standpipe     8776      functional  
2         communal standpipe    34310      functional  
3         communal standpipe    67743  non functional  
4         communal standpipe    19728      functional  
5         communal standpipe     9944     

In [46]:
df[df['installer']=='World Bank']

id  amount_tsh date_recorded      funder  gps_height   installer  \
280     9474       250.0    2013-02-12  World Bank        1343  World Bank   
304    43345         0.0    2012-10-23  World Bank           0  World Bank   
2071   28588         0.0    2012-10-23  World Bank           0  World Bank   
5699   74077         0.0    2012-10-18  World Bank           0  World Bank   
6327   44441         0.0    2012-10-12  World Bank           0  World Bank   
...      ...         ...           ...         ...         ...         ...   
58348  54574         0.0    2012-10-23  World Bank           0  World Bank   
58434  54545       250.0    2013-02-12  World Bank        1311  World Bank   
58681   6335       250.0    2013-02-12  World Bank        1306  World Bank   
58691  67819         0.0    2012-10-12  World Bank           0  World Bank   
58803  44356         0.0    2013-02-06  World Bank        1697  World Bank   

       longitude  latitude           wpt_name  num_private  ... quality_group  \
280    30.017098 -4.332583            Jeshini            0  ...          good   
304    33.430917 -4.389084    Shule Ya Msingi            0  ...         salty   
2071   33.436073 -4.421944        Mwanza Road            0  ...         salty   
5699   33.435268 -4.671744           Kwa Fupe            0  ...         salty   
6327   33.150261 -3.705625         Kwa Maraba            0  ...          good   
...          ...       ...                ...          ...  ...           ...   
58348  33.442457 -4.426592           Kilabuni            0  ...         salty   
58434  30.017351 -4.323850            Mission            0  ...          good   
58681  30.017716 -4.308701         Kwa Thomas            0  ...          good   
58691  33.142605 -3.699442  Kwa Nyamizi Maswa            0  ...          good   
58803  29.784471 -4.445344          Kwa Esrom            0  ...          good   

           quantity quantity_group       source  source_type source_class  \
280    insufficient   insufficient        river   river/lake      surface   
304          enough         enough  machine dbh     borehole  groundwater   
2071         enough         enough  machine dbh     borehole  groundwater   
5699         enough         enough  machine dbh     borehole  groundwater   
6327         enough         enough  machine dbh     borehole  groundwater   
...             ...            ...          ...          ...          ...   
58348        enough         enough  machine dbh     borehole  groundwater   
58434  insufficient   insufficient        river   river/lake      surface   
58681  insufficient   insufficient        river   river/lake      surface   
58691        enough         enough  machine dbh     borehole  groundwater   
58803        enough         enough  machine dbh     borehole  groundwater   

                   waterpoint_type  waterpoint_type_group id_label  \
280             communal standpipe     communal standpipe     9474   
304    communal standpipe multiple     communal standpipe    43345   
2071   communal standpipe multiple     communal standpipe    28588   
5699   communal standpipe multiple     communal standpipe    74077   
6327   communal standpipe multiple     communal standpipe    44441   
...                            ...                    ...      ...   
58348  communal standpipe multiple     communal standpipe    54574   
58434           communal standpipe     communal standpipe    54545   
58681           communal standpipe     communal standpipe     6335   
58691  communal standpipe multiple     communal standpipe    67819   
58803           communal standpipe     communal standpipe    44356   

         status_group  
280        functional  
304        functional  
2071       functional  
5699   non functional  
6327   non functional  
...               ...  
58348      functional  
58434      functional  
58681      functional  
58691  non functional  
58803  non functional  

[95 rows x 42 columns]

In [47]:
df[(df['installer']=='World') & (df['funder']!='World Bank')]

id  amount_tsh date_recorded funder  gps_height installer  \
33726  63910         0.0    2011-04-02  Nethe         141     World   

       longitude  latitude    wpt_name  num_private  ... quality_group  \
33726  38.204463 -6.870355  Kwa Kiwele            0  ...         salty   

           quantity quantity_group        source   source_type source_class  \
33726  insufficient   insufficient  shallow well  shallow well  groundwater   

      waterpoint_type  waterpoint_type_group id_label    status_group  
33726           other                  other    63910  non functional  

[1 rows x 42 columns]

### subvillage

In [48]:
#geographic location
df['subvillage'].isna().sum()

371

In [49]:
df['subvillage'].value_counts()

Madukani       508
Shuleni        506
Majengo        502
Kati           373
Mtakuja        262
              ... 
Kasherero A      1
Chabayege        1
Bunubili         1
Sokony           1
Nyakabumba       1
Name: subvillage, Length: 19287, dtype: int64

In [50]:
list(df['subvillage'].unique())

['Mnyusi B',
 'Nyamara',
 'Majengo',
 'Mahakamani',
 'Kyanyamisa',
 'Moa/Mwereme',
 'Ishinabulandi',
 'Nyawishi Center',
 'Imalauduki',
 'Mkonomre',
 'Mizugo',
 'Ngondombwito',
 'Nkilifa',
 'Omarini',
 'Mwabasabi',
 'Tunzi',
 'Kidudumo',
 'Yeriko',
 'Center',
 'Manyanya',
 'Ibabachegu',
 'Mkanivega',
 'Mkonga Juu',
 'Msasa',
 'Kitereni',
 'Shuleni',
 'Chakahaya',
 'Kiyao',
 'Merali',
 'Karume',
 'Kudipera',
 'Mosheni',
 'Lupanga A',
 'Kilombero B',
 'Afya',
 'Ndanganyika',
 'Baura',
 'Mwanzala',
 'Nyabwai B',
 'Reli B',
 'Kilunduwe',
 'Sokoni',
 'Mwarufyu',
 'Marurani Juu',
 'Isenegeja',
 'Kachulu',
 'Mpandapanda',
 'Mlandege Juu',
 'Ikanga',
 'Msaranga Street',
 'Maporomoko',
 'Isimba',
 'Kagoye B',
 'Dhobi Street',
 'Msufini',
 'Soyekiutu',
 'Ants B',
 "Izimbya 'A'",
 'Nairobi',
 'Nkaloi',
 'Kiganza Centre',
 'Ulkusare',
 'Mzimba',
 'Mfumbu',
 'Mahalule',
 'Kihanga',
 'Iponda',
 'Kisoro',
 'Mtakuja',
 'Lembuka',
 'Mapinduzi',
 'Kalimungoma',
 'Umkituri',
 'Usita',
 'Komoro',
 'Kulasi

In [51]:
df[df['subvillage'] == 'Not Known']

Empty DataFrame
Columns: [id, amount_tsh, date_recorded, funder, gps_height, installer, longitude, latitude, wpt_name, num_private, basin, subvillage, region, region_code, district_code, lga, ward, population, public_meeting, recorded_by, scheme_management, scheme_name, permit, construction_year, extraction_type, extraction_type_group, extraction_type_class, management, management_group, payment, payment_type, water_quality, quality_group, quantity, quantity_group, source, source_type, source_class, waterpoint_type, waterpoint_type_group, id_label, status_group]
Index: []

[0 rows x 42 columns]

### public_meeting

In [79]:
df['public_meeting'].isna().sum()

3334

In [80]:
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

Treat nulls as the majority case, True.

### scheme_management

In [52]:
#who operates the waterpoint (organization/category)
df['scheme_management'].isna().sum()

3877

In [53]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [54]:
df[df['scheme_management'] == "None"]

id  amount_tsh date_recorded funder  gps_height installer  \
23603  23849        50.0    2013-03-18      0         -11         0   

       longitude  latitude      wpt_name  num_private  ... quality_group  \
23603  39.431194 -7.100783  Kwa Nyamtawa            0  ...         salty   

      quantity quantity_group       source  source_type source_class  \
23603   enough         enough  machine dbh     borehole  groundwater   

          waterpoint_type  waterpoint_type_group id_label status_group  
23603  communal standpipe     communal standpipe    23849   functional  

[1 rows x 42 columns]

In [55]:
df[df['funder'] == '0']

id  amount_tsh date_recorded funder  gps_height installer  \
145    28669        50.0    2013-03-21      0          12         0   
152    60983         0.0    2013-03-16      0         -15         0   
393    39749         0.0    2013-03-18      0          28         0   
417    15832        50.0    2013-03-22      0          30         0   
428    50233         0.0    2013-03-12      0          30         0   
...      ...         ...           ...    ...         ...       ...   
59237   2138        59.0    2013-03-19      0          81         0   
59243   3396        50.0    2013-03-16      0         -20         0   
59276  62818        50.0    2013-03-21      0          18         0   
59351  55322        50.0    2013-03-18      0         -19         0   
59387  26640       100.0    2013-03-12      0          25         0   

       longitude  latitude       wpt_name  num_private  ... quality_group  \
145    39.185710 -6.892593       Chekanao            0  ...          good   
152    39.527114 -6.988748      Msikitini            0  ...         salty   
393    39.159887 -6.902548   Kwa Chambuso            0  ...       unknown   
417    39.178404 -6.938013     Ccm Kivule            0  ...          good   
428    39.178849 -6.973206  Ofisi Ya Kata            0  ...       unknown   
...          ...       ...            ...          ...  ...           ...   
59237  39.119109 -6.898919           Kata            0  ...         salty   
59243  39.524021 -6.984802   Kwa Mariwala            0  ...          good   
59276  39.183790 -6.897566   Kwa Mkunduge            0  ...          good   
59351  39.534599 -7.088183      Kwa China            0  ...          good   
59387  39.176480 -6.957098     Kwa Maliba            0  ...          good   

           quantity quantity_group        source   source_type source_class  \
145          enough         enough   machine dbh      borehole  groundwater   
152             dry            dry   machine dbh      borehole  groundwater   
393         unknown        unknown   machine dbh      borehole  groundwater   
417          enough         enough   machine dbh      borehole  groundwater   
428             dry            dry  shallow well  shallow well  groundwater   
...             ...            ...           ...           ...          ...   
59237  insufficient   insufficient   machine dbh      borehole  groundwater   
59243        enough         enough  shallow well  shallow well  groundwater   
59276        enough         enough   machine dbh      borehole  groundwater   
59351        enough         enough   machine dbh      borehole  groundwater   
59387        enough         enough   machine dbh      borehole  groundwater   

          waterpoint_type  waterpoint_type_group id_label    status_group  
145    communal standpipe     communal standpipe    28669      functional  
152                 other                  other    60983  non functional  
393                 other                  other    39749  non functional  
417    communal standpipe     communal standpipe    15832  non functional  
428                 other                  other    50233  non functional  
...                   ...                    ...      ...             ...  
59237  communal standpipe     communal standpipe     2138  non functional  
59243           hand pump              hand pump     3396      functional  
59276  communal standpipe     communal standpipe    62818      functional  
59351  communal standpipe     communal standpipe    55322      functional  
59387  communal standpipe     communal standpipe    26640      functional  

[777 rows x 42 columns]

### scheme_name
### DROP THIS COLUMN
Potential to drop: plethora of names, not enough context to distinguish importance
28,166 nulls limits the usability of the column 

In [56]:
#who operates the waterpoint
#how is this different from scheme_management?
df['scheme_name'].value_counts()

K                                 682
None                              644
Borehole                          546
Chalinze wate                     405
M                                 400
                                 ... 
Mwamihanza  piped water Scheme      1
Leoroma water supply                1
Sasaka village water supply         1
Moroga                              1
Mhembe                              1
Name: scheme_name, Length: 2696, dtype: int64

In [57]:
df['scheme_name'].isna().sum()

28166

In [59]:
unique_scheme = list(df['scheme_name'].unique())
print(len(unique_scheme))
unique_scheme

2697


['Roman',
 nan,
 'Nyumba ya mungu pipe scheme',
 'Zingibali',
 'BL Bondeni',
 'None',
 "wanging'ombe water supply s",
 'Makanj',
 'Kidabu',
 'Mashangwi',
 'Quick wins Program',
 'Komaka mandaka',
 'Sobodo Borehole Scheme',
 'Kitukuni water supply',
 'BL Mwakikoti',
 'Chalinze wate',
 'Mae pipeline',
 'UNDP',
 'Ngana water supplied scheme',
 'Itun',
 'Bomala',
 'Kirua kahe pumping water trust',
 'Misiwa',
 'Mtwango water supplied sche',
 'K',
 'Machumba estate pipe line',
 'SHIMASA',
 'wangama water supply scheme',
 'Shirimatunda Water Supply',
 'Laela group water Supp',
 'Makwale water supplied sche',
 "Nyang'hwale",
 'Njoro Water Supply',
 'Kirua kahe gravity water supply trust',
 'Olgilai pipe line',
 'Mabula mountains spr',
 'Mkongoro One',
 'Maambreni gravity water supply',
 'Mwando water supply',
 'M',
 'Kaisho/Isingiro w',
 "wanging'ombe supply scheme",
 'Tove Mtwango gravity Scheme',
 'Tengeru gravity water supply',
 'Kulasi water supply',
 'Malemb',
 'Tawa',
 'Loruvani gravity 

In [60]:
df['scheme_name'].fillna(value='None', inplace = True)

In [61]:
df['scheme_name'].isna().sum()

0

In [62]:
df['scheme_name'].value_counts()

None                              28810
K                                   682
Borehole                            546
Chalinze wate                       405
M                                   400
                                  ...  
Mwamihanza  piped water Scheme        1
Leoroma water supply                  1
Sasaka village water supply           1
Moroga                                1
Mhembe                                1
Name: scheme_name, Length: 2696, dtype: int64

### Permit

assume that if a permit status is unknown, there is no permit

alternatively: drop 3056 rows (~5% of data)

In [63]:
#if the waterpoint is permitted
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [64]:
df['permit'].isna().sum()

3056

## Column Exploration

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [65]:
#total static head (amount of water available to waterpoint)
df['amount_tsh'].value_counts()

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

In [69]:
df[df['amount_tsh'] == 0.0]['status_group'].value_counts()

functional                 19706
non functional             18885
functional needs repair     3048
Name: status_group, dtype: int64

In [70]:
df['date_recorded']

0        2011-03-14
1        2013-03-06
2        2013-02-25
3        2013-01-28
4        2011-07-13
            ...    
59395    2013-05-03
59396    2011-05-07
59397    2011-04-11
59398    2011-03-08
59399    2011-03-23
Name: date_recorded, Length: 59400, dtype: object

'date_recorded' is stored in yy-mm-dd format

How to encode this as some measure of age in conjunction with year constructed?

Ordinal Encoding?

In [129]:
df['date_recorded_datetime'] = pd.to_datetime(df['date_recorded'])

In [ ]:
df['date_recorded_datetime'] 

In [72]:
#altitude of the well
df['gps_height']

0        1390
1        1399
2         686
3         263
4           0
         ... 
59395    1210
59396    1212
59397       0
59398       0
59399     191
Name: gps_height, Length: 59400, dtype: int64

In [127]:
df['longitude'].value_counts()

0.000000     1812
37.540901       2
33.010510       2
39.093484       2
32.972719       2
             ... 
37.579803       1
33.196490       1
34.017119       1
33.788326       1
30.163579       1
Name: longitude, Length: 57516, dtype: int64

In [128]:
df['latitude'].value_counts()

-2.000000e-08    1812
-6.985842e+00       2
-3.797579e+00       2
-6.981884e+00       2
-7.104625e+00       2
                 ... 
-5.726001e+00       1
-9.646831e+00       1
-8.124530e+00       1
-2.535985e+00       1
-2.598965e+00       1
Name: latitude, Length: 57517, dtype: int64

In [73]:
#name of the waterpoint 
df['wpt_name']

0                        none
1                    Zahanati
2                 Kwa Mahundi
3        Zahanati Ya Nanyumbu
4                     Shuleni
                 ...         
59395     Area Three Namba 27
59396       Kwa Yahona Kuvala
59397                 Mashine
59398                  Mshoro
59399         Kwa Mzee Lugawa
Name: wpt_name, Length: 59400, dtype: object

In [74]:
df['wpt_name'].value_counts()

none                3563
Shuleni             1748
Zahanati             830
Msikitini            535
Kanisani             323
                    ... 
Kindiwiti Camp 7       1
Kwa Kiboko             1
Kwa Mbola Mashoo       1
Bihembe                1
Kwa Nyombe             1
Name: wpt_name, Length: 37400, dtype: int64

'wpt_name' is probably not a useful feature for classification. Some of these are subvillage names (like Shuleni).

In [75]:
#no context
df['num_private'].value_counts()

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

Drop 'num_private'. There is no context provided so no interpretation can be made.

In [76]:
#Geographic water basin
df['basin'].value_counts()

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

In [77]:
#Geographic location
df['subvillage'].value_counts()

Madukani       508
Shuleni        506
Majengo        502
Kati           373
Mtakuja        262
              ... 
Kasherero A      1
Chabayege        1
Bunubili         1
Sokony           1
Nyakabumba       1
Name: subvillage, Length: 19287, dtype: int64

In [82]:
#geographic location
df['region'].value_counts()

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [83]:
#geographic location (coded)
df['region_code'].value_counts()

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [84]:
#geographic location (coded)
df['district_code'].value_counts()

1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64

In [85]:
#geographic location (city?)
df['lga'].value_counts()

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64

In [86]:
#geographic location (ward?)
df['ward'].value_counts()

Igosi              307
Imalinyi           252
Siha Kati          232
Mdandu             231
Nduruma            217
                  ... 
Kihangimahuka        1
Uwanja wa Ndege      1
Uchindile            1
Kapilula             1
Mlimani              1
Name: ward, Length: 2092, dtype: int64

In [88]:
#population around the well
df['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

In [89]:
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [90]:
#drop this column
df['recorded_by'].value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

In [119]:
#how do we treat 0's in construction year?
#age as an ordinal encoded variable to properly treat 0's?
df['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [125]:
df[df['construction_year'] != 0]['construction_year'].describe()
#contruction years range from 1960 to 2013, with 20709 values of 0

count    38691.000000
mean      1996.814686
std         12.472045
min       1960.000000
25%       1987.000000
50%       2000.000000
75%       2008.000000
max       2013.000000
Name: construction_year, dtype: float64

In [92]:
df['extraction_type'].value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [94]:
df['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [95]:
df['extraction_type_class'].value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

Pick out one extraction_type column to incorporate in the model

In [96]:
#how does this differ from scheme_management
df['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [98]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [97]:
df['management_group'].value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

In [99]:
df['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [100]:
df['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

The 'payment' and 'payment_type' columns reflect the same information with the same values per category. Pick one payment column to incorporate.

In [101]:
df['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [102]:
df['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

water_quality and quality_group convey the same information with some slight differences in values. water_quality is more granular in categories

In [104]:
df['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [103]:
df['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

Pick one of the 'quantity' columns.

In [106]:
df['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [107]:
df['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [108]:
df['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [109]:
df['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [110]:
df['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

waterpoint_type and waterpoint_type_group are almost identical except the communal standpipe multiple group in 'waterpoint_type' is included in 'communal standpipe' in 'waterpoint_type_group'

In [114]:
df['id'].value_counts()

2047     1
72310    1
49805    1
51852    1
62091    1
        ..
46396    1
36155    1
34106    1
38200    1
0        1
Name: id, Length: 59400, dtype: int64

In [112]:
df['id_label'].value_counts()

2047     1
72310    1
49805    1
51852    1
62091    1
        ..
46396    1
36155    1
34106    1
38200    1
0        1
Name: id_label, Length: 59400, dtype: int64

Confirmed that no duplicate id appear in the dataset

In [78]:
#FINAL NULLS CHECK
df.isna().sum()

id                          0
amount_tsh                  0
date_recorded               0
funder                      0
gps_height                  0
installer                   0
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                371
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting           3334
recorded_by                 0
scheme_management        3877
scheme_name                 0
permit                   3056
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity  

In [115]:
df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'id_label', 'status_group'],
      dtype='object')

Columns to include in analysis:
amount_tsh (numeric)


Columns to drop:
id
date recorded (age information can be an engineered feature)